In [3]:
# # code required to run on a fresh install or in google colab
# root = "/tmp/XCP-explain"
# ! git clone https://github.com/CPMpy/XCP-explain.git {root}
# ! cd {root} && git checkout ecai24
# ! pip install -r {root}/requirements.txt
# ! pip install cpmpy

# add XCP-explain to the Python path
# import sys
# if root not in sys.path:
#     sys.path.insert(0, root)
root="."

In [4]:
%load_ext autoreload
%autoreload 2
"""
    Some imports used throughout the notebook
"""
import time
import os
from visualize import *

from cpmpy.transformations.normalize import toplevel_list
from factory import *
from read_data import get_data
from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')

import numpy as np
np.set_printoptions(linewidth=90)
# preload solvers
from cpmpy import SolverLookup
names = SolverLookup.solvernames()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-10


## Explaning Optimization problems

In this notebook, we will consider the Nurse Rostering problem as an optimization problem. First we will focus on the following:
- "I **do not like** this solution" &#8592; Find **alternative** optimal or non-dominated solutions 
- "Why is there no **even better** solution?" &#8592; Find what **to change in my model** if I want a better solution?
- "What if **I want Y** in my solution **instead of X**?" &#8592; Find **how to change** the solutions or our optimization model



#### Load the optimization model

Same instance as in the previous notebook.


In [ ]:
instance = os.path.join(root,"Benchmarks/CustomInstance.txt")
data = get_data(instance)
factory = NurseSchedulingFactory(data)

Now as (multi-objective) optimization model! 3 different objectives in nurse rostering:

      - Penalty on positive shift requests

      - Penalty on negative shift requests

      - Penalty of the cover constraints

In [ ]:
model, nurse_view, penalty_on, penalty_off, penalty_cover = factory.get_multi_objective_model()

objectives = [penalty_on, penalty_off, penalty_cover] # penalty_on, penalty_off, penalty_cover are the 3 subobjectives

assert model.solve(solver="ortools", num_workers=8) # you can try different solvers here!
opt_sol = nurse_view.value()
opt_value = model.objective_value()

display(visualize(opt_sol, factory))
print("Total penalty:", model.objective_value())
print("Time to calculate:", model.status().runtime, "s")

## Multiple solutions

- User not satisfied with optimal solution!!

- There could be multiple optimal solutions

- Find (a subset of) them by converting to a decision problem!

In [ ]:
# Convert to decision model enforcing the same objective value and use solveAll (with a limited number of solutions!)
new_model = cp.Model(model.constraints) # init new model
new_model += ... # force objective

solutions_limit = ...
solutions = []

while len(solutions) < solutions_limit:
    new_model.solve(solver="ortools", num_workers=8) # find next solution
    new_model += ~cp.all(nurse_view == nurse_view.value()) # enforce "not same solution"
    solutions.append(nurse_view.value())
    display(visualize(nurse_view.value(), factory))

### Non-Dominated solutions

User not satisfied with optimal solution, considering bad the trade-offs it makes
- Optimize again, considering non-dominated solutions based on sub objectives!
- Non-dominated solution: No other solution will be better (lower penalty) on all objectives...
    - Enforce that at least one of the (3) objectives will have a greater value than **each** found solution

In [ ]:
# We have already loaded the subobjectives: penalty_on, penalty_off, penalty_cover!!
## And we have them stored in the list 'objectives'
non_dom_model = model.copy()

solutions_limit = ...
solutions = []
solution_images = []

while len(solutions) < solutions_limit:
    non_dom_model.solve(solver="ortools", num_workers=8) # find solution by optimizing again
    non_dom_model += ... # dominance constraint
    solutions.append(nurse_view.value())
    solution_images.append([obj.value() for obj in objectives])

for s in solutions:
    display(visualize(s, factory))



In [ ]:
# Create a DataFrame from the solution_images list
solutions_df = pd.DataFrame(
    solution_images,
    columns=['Penalty On', 'Penalty Off', 'Penalty Cover']
)

display(solutions_df)

What is the different trade-off offered from each solution? 

Are some solutions worse than others based on the objective function that we have?

### "Why is there no **even better** solution?" 
&#8592; Find what **to change in my model** if I want a better solution?

- Reduce to UNSAT problem:

  - Add better-than-optimal objective value as constraint: $C \wedge (f(x) < o) \models \bot$

- Then use counterfactual explanations for UNSAT!

In [ ]:
# Find best solution
model.solve(solver="ortools", num_workers=8)
display(visualize(opt_sol, factory))
print("Total penalty:", model.objective_value())


In [ ]:
# what to import and use?

# Convert to UNSAT
constraints = model.constraints
objective_constraint = ...
opt_model = cp.Model(model.constraints)
opt_model += objective_constraint

assert opt_model.solve(solver="ortools", num_workers=8) is False # needs to be false, otherwise there would exist a better solution already
t0 = time.time()
corr_subset = ???(constraints, hard=objective_constraint)
print(len(corr_subset))
print(f"(in {time.time()-t0} seconds)")
visualize_constraints(corr_subset, nurse_view, factory)


Compute and visualise counter-factual solution

In [ ]:
opt_model2 = cp.Model([c for c in model.constraints if ...]) # what constraints do i need to include ?
opt_model2.minimize(model.objective_)  # still need to minimize the objective

assert opt_model2.solve(solver="ortools", num_workers=8) is True
display(visualize(nurse_view.value(), factory))
print("Total penalty:", model.objective_value())

### What if I want Y in my solution instead of X? 
&#8592; Changing the solutions or our model

- Sometimes the user may wants to directly change the solution
    -  due to some constraints or preferences missing
- Or may want to change the weights in the objective
    - to make sure they represent the user's needs



### What if I want Y in my solution instead of X? 
#### Changing the solutions (limited changes)



In [ ]:
# Find optimal solution
model.solve(solver="ortools", num_workers=8)
opt_sol = nurse_view.value()
opt_value = model.objective_value()

display(visualize(opt_sol, factory))
print("Total penalty:", model.objective_value())

Find an assignment that you want to change in the schedule! <- This will be our foil
- Enforce this foil in the model!
- Allow a specific amount of changes (try with different numbers)

Note that 
- shift 'F' is value 0
- shift 'E' is value 1
- shift 'L' is value 2

In [ ]:
Y = ... # The foil ...
mmodel = model.copy()
mmodel += Y 
mmodel += cp.sum(nurse_view != opt_sol) <= ... # allow to make ... changes

assert mmodel.solve(solver="ortools", num_workers=8)
print("Total penalty:", mmodel.objective_value(), "was:", opt_value)

Let's visualize the changes

In [ ]:
style = highlight_changes(nurse_view.value(), opt_sol, factory)
display(style)

### What if I want Y in my solution instead of X? &#8592; 
#### Changing the optimization model!

Use the inverse_optimization method to find what we need to change in our model to enforce a specific foil Y 

In [11]:
instance = os.path.join(root,"Benchmarks/CustomInstance.txt")
data = get_data(instance)
factory = NurseSchedulingFactory(data)
model, nurse_view, penalty_on, penalty_off, penalty_cover = factory.get_multi_objective_model()


In [12]:
assert model.solve(solver="ortools", num_workers=8)
display(visualize(nurse_view.value(), factory))
print("Total penalty: ", model.objective_value())

Total penalty:  1109


Pick now a foil Y from the denied preferences of nurse "Kevin"

In [13]:
nurse = "Kevin"
 
for (w,pref) in zip(*model.objective_.args):
    if nurse in str(pref):
        print('\033[1m' if pref.value() else '\033[0m', f"{pref.value()} \t Penalty:{w} \t{pref} \t")

 False 	 Penalty:2 	Deny Kevin's request to work shift L on Wed 1 	
 False 	 Penalty:2 	Deny Kevin's request to work shift E on Wed 2 	
 True 	 Penalty:2 	Deny Kevin's request to work shift L on Mon 2 	
 False 	 Penalty:2 	Deny Kevin's request not to work shift L on Tue 1 	


In [14]:
desc = "Deny Kevin's request to work shift L on Mon 2"
weight,d_on_fri1 = next((w,pref) for w,pref in zip(*model.objective_.args) if str(pref) == desc)
print(f"{d_on_fri1.value()} \t Penalty:{weight} \t{d_on_fri1}")

True 	 Penalty:2 	Deny Kevin's request to work shift L on Mon 2


In [15]:
foil = {d_on_fri1 : False}  # don't want to have his request for Fri 1 denied!
print("Foil:", foil)
print("\n")

other_prefs = [(w,pref) for w,pref in zip(*model.objective_.args) if nurse in str(pref) and str(pref) != desc]
print(f"{nurse}'s other preferences:")
for w,pref in other_prefs:
    print("- Penalty",w,":",pref)

Foil: {roster[7,7] != 2: False}


Kevin's other preferences:
- Penalty 2 : Deny Kevin's request to work shift L on Wed 1
- Penalty 2 : Deny Kevin's request to work shift E on Wed 2
- Penalty 2 : Deny Kevin's request not to work shift L on Tue 1


In [16]:
from explanations.counterfactual import inverse_optimize

ov = model.objective_value()
new_obj = inverse_optimize(model=model, minimize=True,
                           user_sol = foil,
                           allowed_to_change = set(p[1] for p in other_prefs))
print(f"Done! Found solution with total penalty {new_obj.value()}, was {ov}\n")

# Let's look at the preferences he should enter, to work on Fri 1!
print(f"{nurse}'s new preferences:")
old_w, old_pref = model.objective_.args

for i, arg in enumerate(zip(*new_obj.args)):
    w, pref = arg
    if nurse in str(pref):
        print('\033[1m' if w != old_w[i] else '\033[0m', f"Penalty",w,":",pref)

Done! Found solution with total penalty 1109, was 1109

Kevin's new preferences:
 Penalty 2 : Deny Kevin's request to work shift L on Wed 1
 Penalty 1 : Deny Kevin's request to work shift E on Wed 2
 Penalty 2 : Deny Kevin's request to work shift L on Mon 2
 Penalty 2 : Deny Kevin's request not to work shift L on Tue 1


In [17]:
# compute and visualize new solution
model.minimize(new_obj)
assert model.solve(solver="ortools", num_workers=8)
print("Total penalty: ", model.objective_value())
visualize(nurse_view.value(), factory)

Total penalty:  1109
